In [43]:
import pandas as pd
import pyreadr
import json
import os
import re

In [6]:
os.chdir('/Users/cblanesg/misinformation_socialmedia/data')
os.getcwd()

'/Users/cblanesg/misinformation_socialmedia/data'

## Functions

In [50]:
def obtain_newspapers_id():
    result = pyreadr.read_r('5-analysis/1-input_data/misinformation/data_reg_misinformation.Rda')
    input_reg = result["input_reg"]
    df = input_reg[input_reg['type_misinformation'] == 'media_outlets'][['id_desinformacion', 
                                                               'id_factcheck', 
                                                               'link_desinformacion', 
                                                               'label_desinformacion']].drop_duplicates()
    
    df['media_outlet'] = df['link_desinformacion'].apply(lambda x: re.sub('https://|www.|http://', '',x).split('.')[0])
    return(df)

In [67]:
def obtain_id_newspapers():
    ids = pd.read_excel('3-synthetic_controls/newspapers/0-ids/controls_id.xlsx')[['id_desinformacion', 
                                                                                   'link_desinformacion', 
                                                                                   'link_medio', 
                                                                                   'date_desinformacion']]
    
    ids_control = pd.read_excel('3-synthetic_controls/newspapers/0-ids/controls_true_id.xlsx')[['id_desinformacion', 
                                                                                   'link_desinformacion', 
                                                                                   'link_medio', 
                                                                                   'date_desinformacion']]
    
    id_all = pd.concat([ids, ids_control])
    
    years = []
    months = []
    days = []
    for i in list(id_all.date_desinformacion):
        y, m, d = clean_dates(i)
        years.append(y)
        months.append(m)
        days.append(d)
        
    id_all['year'] = years
    id_all['month'] = months
    id_all['days'] = days
    return(id_all)

In [69]:
def clean_dates(x):
    if pd.isna(x):
        return(None, None, None)
    else:
        x = x.strftime('%Y-%m-%d')
        return(x.split('-')[0], 
              x.split('-')[1], 
              x.split('-')[2])

## Apply Functions

In [70]:
ids = obtain_id_newspapers()

In [71]:
articles_desinformacion = obtain_newspapers_id()

In [72]:
pd.merge(left = articles_desinformacion, 
        right = ids.drop(['link_desinformacion'], axis = 1), 
        on = 'id_desinformacion', 
        how = 'left').to_excel('3-synthetic_controls/newspapers/0-ids/media_ids.xlsx')

In [73]:
pd.merge(left = articles_desinformacion, 
        right = ids.drop(['link_desinformacion'], axis = 1), 
        on = 'id_desinformacion', 
        how = 'left')

,id_desinformacion,id_factcheck,link_desinformacion,label_desinformacion,media_outlet,link_medio,date_desinformacion,year,month,days
0,01839a6d-1c36-3b22-a0c2-2ed9a2633a6e,3dcfc5bb-2f20-3825-bf5e-a0a1175588fc,https://colombianoindignado.com/nino-respondio...,true,colombianoindignado,https://colombianoindignado.com,2019-03-10,2019,03,10
1,02400198-d2d3-38c7-b226-eb98854a750e,fcd455ae-2048-3b98-b583-513b1ea8c799,https://www.dw.com/es/informe-gobierno-alem%C3...,true,dw,NaN,NaT,NaN,NaN,NaN
2,02ae185c-282c-41bc-97f0-7c0503aca187,74d19766-b45e-3145-92a3-789201a6b791,https://albaciudad.org/2020/04/persecucion-y-r...,fake,albaciudad,https://albaciudad.org,NaT,None,None,None
3,02e87747-3140-3eaf-94b1-dde6ad6abc5c,b2d2337c-6760-3a4c-8497-6832967e9e25,https://colombianoindignado.com/new-york-times...,true,colombianoindignado,https://colombianoindignado.com,2019-05-18,2019,05,18
4,032787bb-a1c6-300b-98de-418c660c7f45,f6be5f00-9051-3b67-aeba-0dd9039ccc6b,https://www.elespectador.com/noticias/el-mundo...,true,elespectador,https://www.elespectador.com,2019-02-21,2019,02,21
...,...,...,...,...,...,...,...,...,...,...
494,fcc2bf94-a191-33ee-8310-50e5447d21ea,3ffec203-7bdb-3f05-9739-47e01b1e687d,https://diadelsur.com/el-uso-prolongado-del-ta...,fake,diadelsur,NaN,NaT,NaN,NaN,NaN
495,fd73d687-5922-33a3-bc97-1b40d48d4b84,2aeb692b-9cac-361e-91d9-dcfa3e505ac0,https://www.las2orillas.co/los-5-magistrados-d...,misleading,las2orillas,https://www.las2orillas.co,2019-05-15,2019,05,15
496,fd968c77-e60d-3dcf-aa12-f42a2efcdc58,6eee2604-6459-3178-8e14-fa9493c63cfa,https://www.news24.com/news24/columnists/guest...,fake,news24,NaN,NaT,NaN,NaN,NaN
497,fe0e2aaf-233c-4726-bbb4-c1bb9024fa15,1d60e431-b684-3946-8d14-c3531d1af5b9,https://codigooculto.com/2019/12/cientos-de-av...,fake,codigooculto,https://codigooculto.com,2019-12-30,2019,12,30


In [22]:
with open('3-synthetic_controls/newspapers/3-pre_controls/engagement_controls.json') as f:
    data = json.load(f)

In [38]:
with open('/Users/cblanesg/misinformation_socialmedia/data/3-synthetic_controls/newspapers/1-scrapes/abtc.json') as f:
    data = json.load(f)

In [40]:
pd.DataFrame(data)

,id_article,url_article,newspaper,author,date,body,title
0,4143dd7c-abbf-3776-afc1-be8817cbb109,https://abtc.ng/fiji-coronavirus-cases/,abtc,Local Correspondent,2010-02-11,"Chengdu, China – Scrambled eggs with tomatoes ...",Cheating death: China’s Wuhan coronavirus surv...
1,da6b3bec-88f0-37e7-94c8-e2a9ff6ef9f8,https://abtc.ng/pakistan-coronavirus-news/,abtc,Local Correspondent,2010-02-09,"Chengdu, China – Scrambled eggs with tomatoes ...",Cheating death: China’s Wuhan coronavirus surv...
2,e23e05ef-3a48-3535-853d-c2e7b4b3be9e,https://abtc.ng/autopsy-li-wenliang-coronavirus/,abtc,Local Correspondent,2010-02-08,The death of a Chinese doctor who tried to war...,Li Wenliang: Coronavirus death of Wuhan doctor...
3,89407216-d193-330a-967d-93de1d0b7453,https://abtc.ng/coronavirus-in-jacksonville/,abtc,Local Correspondent,2010-02-07,"Chengdu, China – Scrambled eggs with tomatoes ...",Cheating death: China’s Wuhan coronavirus surv...
4,360c19f2-05bd-3680-af65-a45720d2cf75,https://abtc.ng/coronavirus-cases-houston/,abtc,Local Correspondent,2010-02-03,"Chengdu, China – Scrambled eggs with tomatoes ...",Cheating death: China’s Wuhan coronavirus surv...
...,...,...,...,...,...,...,...
169,95698d43-ed38-3f54-ae01-6cc44fc2a971,https://abtc.ng/ini-edo-birthday-nite-out/,abtc,Local Correspondent,2015-04-24,Birthday nite outBirthday nite out,Ini Edo: Birthday nite out
170,87f49195-bdc5-3b63-b1b4-d1650d44107c,https://abtc.ng/ini-edo-off-we-go-birthday-gro...,abtc,Local Correspondent,2015-04-24,Off we go birthday groove powered by upfront a...,Ini Edo: Off we go birthday groove powered by ...
171,2322a9ee-f902-3828-b483-ea12e71880cf,https://abtc.ng/ini-edo-so-yesterday-was-super...,abtc,Local Correspondent,2015-04-24,So yesterday was super exciting.hanging out wi...,Ini Edo: So yesterday was super exciting.hangi...
172,1f7d4707-fe8a-32e4-bb8b-f69b8b88c4ac,https://abtc.ng/ini-edo-family-whileyouslept-b...,abtc,Local Correspondent,2015-04-23,Family…….#whileyouslept #blood @edojnr @kemzyl...,Ini Edo: Family…….#whileyouslept #blood @edojn...


In [34]:
pd.DataFrame(data[x]).link[0]

'http://www.argumentopolitico.com/2016/07/en-brasil-colgaron-de-un-poste-un.html?m=1'

In [37]:
pd.DataFrame(data[x]).link[2]

'http://www.argumentopolitico.com/2016/07/en-brasil-colgaron-de-un-poste-un.html'